In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType, DateType, BooleanType, DoubleType, ArrayType
import xml.etree.ElementTree as ET
from pyspark import SparkConf
from timeit import default_timer as timer
from pyspark.sql.utils import AnalysisException
import datetime
from time import time
from pyspark.sql.functions import col, lit
import json

In [12]:
from xsd_functions import generate_paths_dict, create_types_dict

In [2]:
jarpath = r"spark-xml_2.11-0.13.0.jar"
path_tm = r"data/knf.tfi.TM.xsd"

In [3]:
spark = SparkSession.builder \
    .appName("SparkApp") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.memoryOverhead", "4g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.yarn.driver.memoryOverhead", "4g") \
    .config("spark.jars",jarpath) \
    .enableHiveSupport().getOrCreate()

- schema 1.1 is available publicly at: https://sprawozdawczosc.knf.gov.pl/podmioty/tfi/okresowe/tfi-tm
- it has a pretty complex strucure as it needs some complementary subschemas with data types description

In [ ]:
# xsd file is read as spark RDD
spark_context = SparkContext.getOrCreate()
xsd_rdd_tm = spark_context.textFile(path_tm).collect()
xsd_rdd_tm

In [8]:
# sub schemas are read as RDD also and boxed in a list
tpp_p = r"data/knf.tpp.xsd"
tpp_liczby_p = r"data/knf.tpp.Liczby.xsd"
tpp_bool_p = r"data/knf.tpp.WskaznikiLogiczne.xsd"
tpp_str_p = r"data/knf.tpp.Teksty.xsd"
tpp_dt_p = r"data/knf.tpp.Daty.xsd"
tpw_w_p = r"data/knf.tpw.Waluty.xsd"


tpp = spark_context.textFile(tpp_p).collect()
tpp_liczby = spark_context.textFile(tpp_liczby_p).collect()
tpp_bool = spark_context.textFile(tpp_bool_p).collect()
tpp_str = spark_context.textFile(tpp_str_p).collect()
tpp_dt = spark_context.textFile(tpp_dt_p).collect()
tpw_w = spark_context.textFile(tpw_w_p).collect()

data_rdds = [tpp_liczby, tpp_bool, tpp_dt]

- Here the create_types_dict and get_data_types functions from xsd_functions.py are used
- product of the function is JSON file types_dict.json that contains mapping of primitive datatypes other than strings

In [9]:
create_types_dict(data_rdds, "types_dict.json")

In [11]:
# reading in JSON that was created above
xpaths_mappings = "types_dict.json"
with open(xpaths_mappings, 'r') as json_file:
    types = json.load(json_file)
types

{'Liczba19.2Typ': 'xs:decimal',
 'Liczba19.4Typ': 'xs:decimal',
 'Liczba19.8Typ': 'xs:decimal',
 'LiczbaNiedodatnia19.2Typ': 'xs:decimal',
 'LiczbaNieujemna19.8Typ': 'xs:decimal',
 'LiczbaCalkowitaNieujemna19Typ': 'xs:decimal',
 'TechnicznyIdentyfikatorElementu': 'xs:decimal',
 'WskaznikLogiczny0lub1Typ': 'xs:boolean',
 'WskaznikLogicznytruelubfalseTyp': 'xs:boolean',
 'WskaznikLogiczny0albofalseTyp': 'xs:boolean',
 'WskaznikLogiczny1albotrueTyp': 'xs:boolean',
 'WskaznikLogiczny0lub1albotruelubfalseTyp': 'xs:boolean',
 'DataPrzedzial1Typ': 'xs:date',
 'DataPrzedzial2Typ': 'xs:date',
 'DataPrzedzial3Typ': 'xs:date'}

- Here generate_paths_dict function is using get_types and build_xpaths functions from xsd_functions.py to recurrently build xpaths
- Python's xml library is levraged here to read basic xml structure
- The result is tm_xpaths_maping.json that mapps xpaths data points from xml file to primitive datatypes

In [18]:
xpaths_dict = generate_paths_dict("tm_xpaths_maping.json", "SprawozdanieTyp", types, xsd_rdd_tm)

# showing 5 exaple entries in xpaths_dict dictionary
list(xpaths_dict.items())[:5]

[('Naglowek.IdentyfikatorZarzadzajacego', 'xs:string'),
 ('Naglowek.NazwaZarzadzajacego', 'xs:string'),
 ('Naglowek.IdentyfikatorDostawcyDanych', 'xs:string'),
 ('Naglowek.WalutaSprawozdania', 'xs:string'),
 ('Naglowek.OkresSprawozdawczy.DataOd', 'xs:date')]

- some complex files requires additional schemas other than just simple datatypes
- function generate_paths_dict can accomodate that by adding optional argument as list of RDDs of subschemas

In [19]:
p = r"data/knf.fi.Lokaty.xsd"
p_abs = r"data/knf.abs.Aktywa.xsd"
p_abs_id = r"data/knf.abs.IdentyfikacjaSkladnika.xsd"

xsd_rdd = spark_context.textFile(p).collect()
xsd_rdd2 = spark_context.textFile(p_abs).collect()
xsd_rdd3 = spark_context.textFile(p_abs_id).collect()

In [24]:
xpaths_dict = generate_paths_dict("tm_xpaths_maping.json", "SprawozdanieTyp", types, xsd_rdd, [xsd_rdd2, xsd_rdd3])
xpaths_dict

IdentyfikacjaEmitentaTyp


{'Naglowek.IdentyfikatorZarzadzajacego': 'xs:string',
 'Naglowek.NazwaZarzadzajacego': 'xs:string',
 'Naglowek.IdentyfikatorFunduszuSubfunduszu': 'xs:string',
 'Naglowek.NazwaFunduszuSubfunduszu': 'xs:string',
 'Naglowek.IdentyfikatorDostawcyDanych': 'xs:string',
 'Naglowek.DzienWyceny': 'xs:date',
 'Naglowek.WalutaSprawozdania': 'xs:string',
 'Naglowek.OkresSprawozdawczy.DataOd': 'xs:date',
 'Naglowek.OkresSprawozdawczy.DataDo': 'xs:date',
 'Naglowek.OkresSprawozdawczy.Rok': 'xs:string',
 'Naglowek.OkresSprawozdawczy.Kwartal': 'xs:string',
 'Naglowek.OkresSprawozdawczy.Miesiac': 'xs:string',
 'Naglowek.CzyKorekta': 'xs:boolean',
 'Naglowek._CzyFunduszWLikwidacji': 'xs:boolean',
 'Dane.Aktywa.SkladnikAktywow.IdentyfikacjaSkladnika.Identyfikator': 'xs:string',
 'Dane.Aktywa.SkladnikAktywow.IdentyfikacjaSkladnika.Nazwa': 'xs:string',
 'Dane.Aktywa.SkladnikAktywow.IdentyfikacjaSkladnika.Specyfikacja.KategoriaAktywow': 'xs:string',
 'Dane.Aktywa.SkladnikAktywow.IdentyfikacjaSkladnika.Specy